In [52]:
import os
import pdb
import json
import numpy as np
from tqdm import tqdm

# ConceptNet word embeddings -  better than GLOVE!
WORD_EMBED='/home/brigit/sandbox/sg2im_brigit/word_embeddings/numberbatch-en-19.08.txt'
#WORD_EMBED='/home/brigit/sandbox/sg2im_brigit/word_embeddings/glove.6B.300d.txt'
src = 'conceptnet'
#src = 'glove300d'
JSON_VOCAB='/home/brigit/sandbox/sg2im_brigit/datasets/vg/vocab.json'

# load word embeddings
embeddings_dict = {}
head = True
with open(WORD_EMBED, 'r') as f:
  for line in tqdm(f):
    if head:
      head = False
      continue
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], "float32")
    embeddings_dict[word] = vector

516783it [00:26, 19172.71it/s]


In [53]:
# load VG vocab for objects and relationships
with open(JSON_VOCAB) as f:
  vocab = json.load(f)

del vocab['object_name_to_idx']['__image__']
del vocab['pred_name_to_idx']['__in_image__']
objects = list(vocab['object_name_to_idx'].keys())
predicates = list(vocab['pred_name_to_idx'].keys())
predicates_clean = []

#clean predicate list - to deal with cases like 'walking on'
for p in predicates:
   vals = p.split()
   if len(vals) <= 2: # 'walking on', 'eating'
     predicates_clean.append(vals[0])
   elif len(vals) == 3: # 'on top of'
     predicates_clean.append(vals[1])

In [54]:
f = open(src + '_synonyms.txt', "w")

dist_dict = {}
for obj in objects:
  if obj in embeddings_dict:
    dist = []
    for other_obj in objects:
      if other_obj in embeddings_dict:
        d = np.dot(embeddings_dict[obj]/np.linalg.norm(embeddings_dict[obj]), embeddings_dict[other_obj]/np.linalg.norm(embeddings_dict[other_obj]))
        # normalize distance
        d = (d+1)/2
        dist.append(d)
    sort_idx = np.argsort(dist)[::-1]
    sort_dist = np.around(np.array(dist)[sort_idx],3)
    sort_objs = np.array(objects)[sort_idx]
    objs_dist = list(zip(sort_objs[1:10], sort_dist[1:10]))
    print(obj, ':', objs_dist, '\n')
    l = [ "%s (%s)" % x for x in objs_dist ]
    f.write(obj + ': ' + ', '.join(l) + '\n')  
    dist_dict[obj] = objs_dist
  else:
    print('skipping ', obj)

for obj in predicates_clean:
  if obj in embeddings_dict:
    dist = []
    for other_obj in predicates_clean:
      if other_obj in embeddings_dict:
        d = np.dot(embeddings_dict[obj]/np.linalg.norm(embeddings_dict[obj]), embeddings_dict[other_obj]/np.linalg.norm(embeddings_dict[other_obj]))
        # normalize distance
        d = (d+1)/2
        dist.append(d)
    sort_idx = np.argsort(dist)[::-1]
    sort_dist = np.around(np.array(dist)[sort_idx],3)
    sort_objs = np.array(objects)[sort_idx]
    objs_dist = list(zip(sort_objs[1:10], sort_dist[1:10]))
    print(obj, ':', objs_dist, '\n')
    l = [ "%s (%s)" % x for x in objs_dist ]
    f.write(obj + ': ' + ', '.join(l) + '\n')  
    dist_dict[obj] = objs_dist
  else:
    print('skipping ', obj)
f.close()

out_file = open(src + '_synonyms.json', "w") 
json.dump(dist_dict, out_file, indent = 5 )
out_file.close()

window : [('door', 0.805), ('windshield', 0.74), ('curtain', 0.735), ('glass', 0.733), ('roof', 0.72), ('wall', 0.707), ('ceiling', 0.702), ('mirror', 0.686), ('tile', 0.67)] 

tree : [('leaf', 0.738), ('branch', 0.736), ('wood', 0.736), ('bush', 0.68), ('trunk', 0.677), ('plant', 0.67), ('grass', 0.651), ('flower', 0.647), ('pole', 0.64)] 

man : [('woman', 0.855), ('boy', 0.766), ('person', 0.742), ('girl', 0.716), ('lady', 0.702), ('people', 0.677), ('child', 0.601), ('player', 0.593), ('stand', 0.58)] 

shirt : [('jacket', 0.851), ('coat', 0.752), ('hat', 0.735), ('pant', 0.731), ('jean', 0.726), ('cap', 0.683), ('logo', 0.679), ('sock', 0.678), ('stripe', 0.677)] 

wall : [('fence', 0.769), ('ceiling', 0.764), ('railing', 0.752), ('brick', 0.717), ('window', 0.707), ('tile', 0.703), ('roof', 0.68), ('curtain', 0.679), ('stone', 0.668)] 

person : [('people', 0.759), ('man', 0.742), ('woman', 0.732), ('girl', 0.643), ('child', 0.63), ('lady', 0.605), ('player', 0.6), ('boy', 0.59),

bush : [('tree', 0.68), ('wood', 0.62), ('leaf', 0.619), ('branch', 0.61), ('grass', 0.609), ('hill', 0.6), ('flower', 0.6), ('cabinet', 0.595), ('fence', 0.592)] 

boy : [('girl', 0.869), ('child', 0.805), ('man', 0.766), ('woman', 0.664), ('person', 0.59), ('lady', 0.59), ('giraffe', 0.575), ('dog', 0.575), ('horse', 0.571)] 

helmet : [('cap', 0.762), ('hat', 0.76), ('motorcycle', 0.729), ('bike', 0.702), ('jacket', 0.7), ('glove', 0.683), ('glasses', 0.672), ('sunglass', 0.669), ('shirt', 0.65)] 

street : [('sidewalk', 0.826), ('road', 0.818), ('building', 0.673), ('floor', 0.643), ('hill', 0.638), ('track', 0.634), ('brick', 0.634), ('house', 0.628), ('wall', 0.627)] 

field : [('ground', 0.703), ('grass', 0.69), ('ball', 0.652), ('line', 0.644), ('fence', 0.637), ('track', 0.626), ('background', 0.613), ('stripe', 0.611), ('dirt', 0.604)] 

bottle : [('glass', 0.756), ('container', 0.751), ('cup', 0.699), ('glasses', 0.685), ('box', 0.667), ('water', 0.665), ('bag', 0.653), ('ca

cup : [('bowl', 0.77), ('bottle', 0.699), ('pot', 0.688), ('glass', 0.676), ('container', 0.662), ('plate', 0.622), ('water', 0.613), ('leaf', 0.609), ('leg', 0.608)] 

elephant : [('giraffe', 0.79), ('zebra', 0.732), ('animal', 0.713), ('bear', 0.69), ('cow', 0.674), ('horse', 0.643), ('horn', 0.639), ('paw', 0.619), ('dog', 0.613)] 

cabinet : [('shelf', 0.754), ('chair', 0.664), ('table', 0.66), ('door', 0.656), ('knob', 0.652), ('bench', 0.649), ('ceiling', 0.646), ('box', 0.641), ('house', 0.639)] 

coat : [('jacket', 0.88), ('shirt', 0.752), ('hat', 0.733), ('glove', 0.681), ('pant', 0.665), ('cap', 0.657), ('helmet', 0.645), ('bag', 0.637), ('hair', 0.636)] 

mountain : [('hill', 0.824), ('ski', 0.739), ('snow', 0.685), ('rock', 0.673), ('top', 0.662), ('sky', 0.644), ('bike', 0.634), ('bear', 0.632), ('road', 0.628)] 

giraffe : [('zebra', 0.814), ('elephant', 0.79), ('animal', 0.699), ('cow', 0.666), ('bear', 0.646), ('horn', 0.62), ('sheep', 0.611), ('paw', 0.605), ('stripe',

pot : [('bowl', 0.701), ('cup', 0.688), ('container', 0.676), ('sink', 0.647), ('plate', 0.642), ('table', 0.636), ('knob', 0.626), ('flower', 0.622), ('bottle', 0.611)] 

apple : [('orange', 0.694), ('banana', 0.683), ('tree', 0.629), ('leaf', 0.578), ('logo', 0.577), ('pizza', 0.576), ('cow', 0.572), ('bush', 0.57), ('sheep', 0.568)] 

basket : [('ball', 0.705), ('box', 0.685), ('bag', 0.667), ('container', 0.667), ('towel', 0.643), ('table', 0.637), ('bench', 0.633), ('pillow', 0.628), ('bowl', 0.625)] 

plane : [('bus', 0.683), ('boat', 0.666), ('kite', 0.666), ('wing', 0.655), ('train', 0.651), ('sky', 0.642), ('seat', 0.641), ('ground', 0.638), ('engine', 0.633)] 

key : [('door', 0.674), ('knob', 0.668), ('button', 0.651), ('part', 0.625), ('top', 0.615), ('number', 0.611), ('finger', 0.604), ('window', 0.59), ('word', 0.583)] 

tie : [('neck', 0.664), ('shirt', 0.658), ('wire', 0.651), ('stripe', 0.63), ('hat', 0.625), ('cap', 0.619), ('sock', 0.615), ('flag', 0.604), ('leg', 0

In [51]:
dist_dict



{'window': [('door', 0.835),
  ('glass', 0.777),
  ('roof', 0.77),
  ('windshield', 0.74),
  ('ceiling', 0.737),
  ('floor', 0.733),
  ('frame', 0.722),
  ('curtain', 0.708),
  ('wall', 0.703)],
 'tree': [('flower', 0.73),
  ('leaf', 0.718),
  ('wood', 0.713),
  ('trunk', 0.709),
  ('grass', 0.707),
  ('plant', 0.677),
  ('roof', 0.677),
  ('banana', 0.675),
  ('fence', 0.666)],
 'man': [('woman', 0.85),
  ('person', 0.822),
  ('boy', 0.81),
  ('girl', 0.775),
  ('hand', 0.741),
  ('face', 0.726),
  ('people', 0.722),
  ('back', 0.708),
  ('dog', 0.702)],
 'shirt': [('jacket', 0.851),
  ('hat', 0.766),
  ('coat', 0.731),
  ('hair', 0.721),
  ('tie', 0.715),
  ('logo', 0.714),
  ('cap', 0.707),
  ('helmet', 0.707),
  ('glasses', 0.702)],
 'wall': [('street', 0.811),
  ('fence', 0.722),
  ('floor', 0.721),
  ('glass', 0.719),
  ('ceiling', 0.711),
  ('roof', 0.711),
  ('building', 0.708),
  ('brick', 0.706),
  ('window', 0.703)],
 'person': [('man', 0.822),
  ('woman', 0.809),
  ('people